In [7]:
import tensorflow as tf
import os

**load the data**

In [ ]:
currentDir = os.getcwd()

Train_filePath = os.path.join(currentDir, "Flower Classification Dataset/train")
Test_filePath = os.path.join(currentDir, "Flower Classification Dataset/test")
Validation_filePath = os.path.join(currentDir, "Flower Classification Dataset/valid")

Train_data = tf.keras.preprocessing.image_dataset_from_directory(
    Train_filePath,
    labels = 'inferred',
    image_size = (224,224),
    batch_size = 102,
    #validation_split = 0
)

Test_data = tf.keras.preprocessing.image_dataset_from_directory(
    Test_filePath,
    labels = 'inferred',
    image_size = (224,224),
    batch_size = 102,
    #validation_split = 0
)

Validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    Validation_filePath,
    labels = 'inferred',
    image_size = (224,224),
    batch_size = 102,
    #validation_split = 0
)

#Split the dataset into input and target
# Input_validation= Validation_data.map(lambda image, lable: image)
# Target_validation= Validation_data.map(lambda image, lable: lable)

Found 4102 files belonging to 102 classes.
Found 2042 files belonging to 102 classes.
Found 2045 files belonging to 102 classes.


**Outline the model**

In [9]:
input_size = 50176
output_size = 102
hidden_layer_size = 128 #TODO: figure out the exact value

model = tf.keras.Sequential([
    # ReScale pixel values in the band [0, 1] 
    tf.keras.Input(shape=(224, 224, 3)),
    tf.keras.layers.Rescaling(1./255),

    # Augmentation (Adding randomization to the images)
    # ONLY USED IN TRAINING
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.2),
    
    # Conv 1
    tf.keras.layers.Conv2D(
        32, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001),
        padding='same'
    ), #TODO: Check if the combination of L1 and L2 works better for all layers
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    # Conv 2
    tf.keras.layers.Conv2D(
        64, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001),
        padding='same'
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    # Conv 3
    tf.keras.layers.Conv2D(
        64, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001),
        padding='same'
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    # Conv 4
    tf.keras.layers.Conv2D(
        128, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001),
        padding='same'
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    # Flattened Inputs
    tf.keras.layers.GlobalAveragePooling2D(), # Better than normal Flattening, reduces the dimensionality of feature maps
        #tf.keras.layers.Flatten(),
    
    # Dense Layers (Fully connected layers (FC Layers))
    tf.keras.layers.Dense(
        hidden_layer_size, 
        activation = 'relu', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    #TODO: Check if dropping neurons actually prevents overfitting 
    #TODO: We can test other techniques besides dropping 
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(        
        hidden_layer_size, 
        activation = 'relu', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    tf.keras.layers.Dropout(0.3),

    # Didn't specify "softmax" activation function in the last layer 
    # to use the right order of opertions for loss calculations
    tf.keras.layers.Dense(output_size) 
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip_1 (RandomFlip)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_1               │ (None, 224, 224, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom_1 (RandomZoom)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_contrast_1               │ (None, 224, 224, 3)    │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 128)            │             

 Total params: 177,510 (693.40 KB)

 Trainable params: 176,934 (691.15 KB)

 Non-trainable params: 576 (2.25 KB)

**Choose the optimizer and the loss function**

In [10]:
lossFunction = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) # Internally applies "softmax" if "from_logits = True"

#TODO: Add early stopping
model.compile(optimizer= 'adam' ,loss=lossFunction, metrics= ['accuracy'])

**Training**

In [16]:
epoch_num= 10

model.fit(Train_data, validation_data= Validation_data, epochs= epoch_num, verbose= 2)

Epoch 1/10
41/41 - 158s - 4s/step - accuracy: 0.0473 - loss: 4.9219 - val_accuracy: 0.0469 - val_loss: 4.9572
Epoch 2/10
41/41 - 146s - 4s/step - accuracy: 0.1007 - loss: 4.4550 - val_accuracy: 0.0313 - val_loss: 4.8511
Epoch 3/10
41/41 - 147s - 4s/step - accuracy: 0.1275 - loss: 4.1276 - val_accuracy: 0.0181 - val_loss: 5.0887
Epoch 4/10
41/41 - 142s - 3s/step - accuracy: 0.1455 - loss: 3.8926 - val_accuracy: 0.0274 - val_loss: 5.4473
Epoch 5/10
41/41 - 162s - 4s/step - accuracy: 0.1716 - loss: 3.7190 - val_accuracy: 0.0469 - val_loss: 6.3921
Epoch 6/10
41/41 - 153s - 4s/step - accuracy: 0.1916 - loss: 3.5874 - val_accuracy: 0.0567 - val_loss: 5.7613
Epoch 7/10
41/41 - 149s - 4s/step - accuracy: 0.2111 - loss: 3.4557 - val_accuracy: 0.0562 - val_loss: 6.0960
Epoch 8/10
41/41 - 155s - 4s/step - accuracy: 0.2199 - loss: 3.3832 - val_accuracy: 0.0484 - val_loss: 6.0290
Epoch 9/10
41/41 - 145s - 4s/step - accuracy: 0.2445 - loss: 3.2924 - val_accuracy: 0.0636 - val_loss: 5.5962
Epoch 10/1

**Evaluate the model on test data**

In [ ]:
# test_loss, test_accuracy = model.evaluate(Test_data)
# print(f"Test accuracy: {test_accuracy}")